In [1]:
%config InlineBackend.figure_format = 'retina'
%pylab inline
### Libraries 
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import geopandas as gpd
import statsmodels.api as sm
import libpysal
import seaborn as sns
import matplotlib.patches as patches
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.gridspec as gridspec

Populating the interactive namespace from numpy and matplotlib


In [2]:
################################### cities ###################################################
dfc = gpd.read_file('../data_revision/cities/all/gdfCities.shp')
########################### ALL CITIES #############################
treshold = 0.85
di = 0.1
cols = ['city', 'ep0','ep0F', 'ep1F', 
        'deltaExposure0', 'deltaExposure1', 
        'deltaNDVIGlobal0','deltaNDVIGlobal1', 'deltaNDVILocal']
dff=pd.DataFrame(columns=cols)

In [3]:
betas_day_ = np.array([3.64209653, -4.67311115, -1.16021593,  0.62546181])
betas_night_ =  np.array([2.23640437, -2.68122998, -0.59718829,  0.79733414])

In [3]:
df_res_day = pd.read_csv("../data_revision/coefficients/dfBetas_day.csv")
df_res_night = pd.read_csv("../data_revision/coefficients/dfBetas_night.csv")
betas_night_ = np.mean(df_res_night, axis=0).values
betas_day_ = np.mean(df_res_day, axis=0).values

In [11]:
def incrNDVI(treshold, di, gdf_city):
    mi = treshold - di
    #dfc1 = dfc[dfc.index==idx]
    gdf_city['NDVI_']=gdf_city['NDVI']
    
    idxs1 = np.where(gdf_city['NDVI_']<mi)
    idxs1 = list(idxs1[0])
    
    idxs2 = np.where((gdf_city['NDVI']>=mi) & ((gdf_city['NDVI']<treshold)))
    idxs2 = list(idxs2[0])
                     
    gdf_city['DN'] = 0
    gdf_city['DN'][idxs2]=0.85-gdf_city['NDVI'][idxs2]
                     
    gdf_city['NDVI_'][idxs2] = gdf_city['NDVI_'][idxs2]+gdf_city['DN'][idxs2]
    gdf_city['NDVI_'][idxs1] = gdf_city['NDVI_'][idxs1]+di
    
    return gdf_city

def estimate(target_var, predictor_vars, gdf_city_c, w_test, betas_):
    y_test = gdf_city_c[target_var].values.reshape(-1, 1)
    w_test = libpysal.weights.lag_spatial(w_, y_test)
    x_test = gdf_city_c[predictor_vars].values
    constant = sm.add_constant(np.hstack((x_test,np.array(w_test).reshape(-1,1))))
    ypred_  = np.sum(constant * (betas_.T), axis=1).reshape((-1, 1))
    return ypred_

def exposure(df, col1, col2, pop):
    return (np.sum(df[col1]*df[pop])+np.sum(df[col2]*df[pop]))/2

def norm01(df, column):
    return (df[column] - df[column].min()) / (df[column].max() - df[column].min())

def estimateExposure(predictor_vars,predictor_vars1, target_vars, gdf_city, w_, betas_day, betas_night):
    y_pred_day    = estimate(target_vars[0], predictor_vars, gdf_city, w_, betas_day)
    y_pred_night  = estimate(target_vars[1], predictor_vars, gdf_city, w_, betas_night)
    y_pred_day1 = estimate(target_vars[0], predictor_vars1, gdf_city, w_, betas_day)
    y_pred_night1 = estimate(target_vars[1], predictor_vars1, gdf_city, w_, betas_night)
    
    gdf_city['y_day1']=y_pred_day1
    gdf_city['y_night1']=y_pred_night1

    gdf_city['y_day']=y_pred_day
    gdf_city['y_night']=y_pred_night
    ep1 = exposure(gdf_city, 'y_day1', 'y_night1', 'pop')
    ep = exposure(gdf_city, 'y_day', 'y_night', 'pop')
    
    gdf_city['ep1'] = 0.5*gdf_city['pop']*(gdf_city['y_day1']+gdf_city['y_night1'])
    gdf_city['ep'] = 0.5*gdf_city['pop']*(gdf_city['y_day']+gdf_city['y_night'])
    
    gdf_city['ed1'] = 0.5*(gdf_city['y_day1']+gdf_city['y_night1'])
    gdf_city['ed'] = 0.5*(gdf_city['y_day']+gdf_city['y_night'])
    
    return ep, ep1, gdf_city

In [9]:
target_vars = ['hot_days', 'hot_nights']
predictor_vars = ['NDVI',  'dist_n']
predictor_vars1 = ['NDVI_', 'dist_n']

In [29]:
dis = [0.1, 0.2, 0.3, 0.4, 0.5]
cols = ['city', 'ep0','ep0F', 'ep1F', 
        'deltaExposure0', 'deltaExposure1', 
        'deltaNDVIGlobal0','deltaNDVIGlobal1', 'deltaNDVILocal', 'di']
dff=pd.DataFrame(columns=cols)

for di in dis:
    for i in range(0, 200):
        fout = '../data_revision/GEE_dataframes/gdf_%d.shp'%i
        gdf_city  = gpd.read_file(fout)
        
        gdf_city  = gpd.read_file(fout)
        gdf_city['hot_days']=gdf_city['hot_days']/11
        gdf_city['hot_nights']=gdf_city['hot_nights']/11
        gdf_city['dist']=-gdf_city['dist']
        gdf_city=gdf_city[gdf_city['dist']>0]
        gdf_city['dist_n'] = 1/(gdf_city['dist']**2)
        gdf_city['dist_n'] = norm01(gdf_city, 'dist_n')
        gdf_city=gdf_city.reset_index(drop=True)
        gdf_city = gdf_city[gdf_city['NDVI']>=-1]
        gdf_city = gdf_city[gdf_city['NDBI']>=-1].reset_index(drop=True)
        
        
        dfc1 = dfc[dfc.index==i]
        gdf_city = incrNDVI(treshold, di, gdf_city)
        if len(gdf_city['NDVI_'].unique())==1:continue
        w_ = libpysal.weights.KNN.from_dataframe(gdf_city, k=8)
        w_.transform = 'r'
        ep, ep1, gdf_city  = estimateExposure(predictor_vars,predictor_vars1, target_vars, gdf_city, w_,
                                   betas_day_, betas_night_)
        Ep0 = 100*(ep1-ep)/ep
        DV = 100*(gdf_city['NDVI_'].sum()-gdf_city['NDVI'].sum())/(gdf_city['NDVI'].sum())



        ######## Find optimal ################
        tmp = gdf_city[['pop','NDVI','geometry']]
        tmp = tmp.sort_values(['pop'], ascending=False)
        tmp['cs']   = np.cumsum(tmp['pop'])
        tmp['css']  = tmp['cs']/tmp['pop'].sum()
        tmp=tmp[tmp['css']<=0.8]                              ###### Focus where the population live

        ep11 = ep1+1
        incr = 0.01
        tmp['NDVI_']=tmp['NDVI']

        while (ep11 >= ep1) and tmp['NDVI_'].min()<0.85:

            tmp['NDVI_']=tmp['NDVI']+di+incr
            j = list(np.where(tmp['NDVI_']>0.85)[0])
            idx = tmp.index[j]
            tmp['NDVI_'][idx]=0.85
            gdf_city['NDVI_'] = gdf_city['NDVI']
            gdf_city['NDVI_'][tmp.index]=tmp['NDVI_'][tmp.index]

            epp, ep11, gdf_city  = estimateExposure(predictor_vars, predictor_vars1, target_vars, gdf_city, w_,
                                                   betas_day_, betas_night_)
            Ep1 = 100*(ep11-ep)/ep
            incr+=0.01

        DV1 = 100*(gdf_city['NDVI_'].sum()-gdf_city['NDVI'].sum())/(gdf_city['NDVI'].sum())
        data = [dfc['UC_NM_MN'][i], ep, ep1, ep11, Ep0, Ep1, DV, DV1, di+incr, di]
        dff = dff.append(pd.DataFrame(columns=cols, data=[data]), ignore_index=True)
        print(data[0], data[0:1], data[6:])

Guangzhou ['Guangzhou'] [22.418835006717483, 6.751968519426004, 0.14, 0.1]
Los Angeles ['Los Angeles'] [29.859159101067434, 9.307022701846138, 0.14, 0.1]
New York ['New York'] [10.13323377878236, 4.614223107447679, 0.14, 0.1]
Tokyo ['Tokyo'] [16.227620208934926, 6.140417415874879, 0.14, 0.1]
Jakarta ['Jakarta'] [14.74071230433106, 4.986738084197053, 0.14, 0.1]
Chicago ['Chicago'] [15.596577125758328, 6.645854733447868, 0.14, 0.1]
Dallas ['Dallas'] [21.01478456905699, 7.336803396945974, 0.14, 0.1]
Houston ['Houston'] [16.936055049488107, 5.316146078566873, 0.14, 0.1]
Shanghai ['Shanghai'] [21.71787546589059, 8.446395328232601, 0.14, 0.1]
Dhaka ['Dhaka'] [19.619540150464108, 7.4988811705531075, 0.14, 0.1]
Osaka [Kyoto] ['Osaka [Kyoto]'] [13.797626364297523, 4.3536689719145345, 0.14, 0.1]
Miami ['Miami'] [20.33218932670188, 6.354138640162443, 0.14, 0.1]
Kolkata ['Kolkata'] [20.106213551206647, 6.997951079014266, 0.14, 0.1]
Nagoya ['Nagoya'] [14.969575321926706, 7.0838598490755595, 0.14, 0

Pune ['Pune'] [33.468750379432656, 7.977720437473191, 0.14, 0.1]
Takasaki [Maebashi] ['Takasaki [Maebashi]'] [15.268251847081782, 9.261608908859072, 0.14, 0.1]
Nanjing ['Nanjing'] [20.452688730910857, 5.194612372830861, 0.14, 0.1]
Tegal ['Tegal'] [17.663664412846032, 7.012093510205398, 0.14, 0.1]
Bengaluru ['Bengaluru'] [27.98279172457859, 4.747080738991015, 0.14, 0.1]
Xi'an ["Xi'an"] [22.73538279722437, 7.946390080190424, 0.14, 0.1]
Kollam ['Kollam'] [15.752913881761755, 10.620004859143878, 0.14, 0.1]
Austin ['Austin'] [20.070416986607636, 6.971281023363851, 0.14, 0.1]
Changzhou ['Changzhou'] [21.491407765593177, 11.902085019200229, 0.14, 0.1]
Zhengzhou ['Zhengzhou'] [20.277704948491568, 5.790112880987602, 0.14, 0.1]
Qingdao ['Qingdao'] [20.47595781883008, 5.814567360685193, 0.14, 0.1]
Alexandria ['Alexandria'] [23.395958486217097, 5.0153544367727765, 0.14, 0.1]
Guadalajara ['Guadalajara'] [29.219683937290824, 7.862053544902399, 0.14, 0.1]
Belo Horizonte ['Belo Horizonte'] [17.3282243

Buenos Aires ['Buenos Aires'] [36.00289945886053, 13.493417720372241, 0.26, 0.2]
Suzhou ['Suzhou'] [43.38981238877125, 19.32314165811647, 0.27, 0.2]
Seattle ['Seattle'] [19.791591469303853, 9.752368609481959, 0.26, 0.2]
Moscow ['Moscow'] [23.078235984080198, 6.972543375114393, 0.26, 0.2]
London ['London'] [30.782662443421735, 10.482697300804704, 0.26, 0.2]
Tijuana ['Tijuana'] [60.67843582062038, 15.505369130677394, 0.27, 0.2]
Surabaya ['Surabaya'] [32.48752488116617, 13.385576291270482, 0.26, 0.2]
Johannesburg ['Johannesburg'] [42.87978669864602, 11.976199533524253, 0.27, 0.2]
Paris ['Paris'] [30.433596263114175, 9.74325389606453, 0.26, 0.2]
Melbourne ['Melbourne'] [44.232830330327715, 16.429861958914778, 0.26, 0.2]
Cairo ['Cairo'] [80.2168841963991, 16.332784077002422, 0.27, 0.2]
Yogyakarta ['Yogyakarta'] [24.088124239279185, 13.448779635504255, 0.29000000000000004, 0.2]
Washington D.C. ['Washington D.C.'] [14.993933317182083, 7.628410457308057, 0.28, 0.2]
Philadelphia ['Philadelphia'

Jeddah ['Jeddah'] [260.2737168912154, 91.58999151361756, 0.27, 0.2]
Ciudad Juárez ['Ciudad Juárez'] [113.32424466587024, 33.3451517872337, 0.27, 0.2]
Tasikmalaya ['Tasikmalaya'] [21.33306311210417, 8.908037915198092, 0.28, 0.2]
Jinan ['Jinan'] [33.343931734740465, 8.464538688918454, 0.26, 0.2]
Salt Lake City ['Salt Lake City'] [46.83439585569037, 14.980297297213697, 0.27, 0.2]
Kampala ['Kampala'] [31.90408385835129, 11.689196802033205, 0.26, 0.2]
Saint Petersburg ['Saint Petersburg'] [20.939946686038077, 7.602456538404586, 0.26, 0.2]
Yangon ['Yangon'] [51.70221369895076, 12.212723715189457, 0.27, 0.2]
Guruvayur ['Guruvayur'] [33.445613270542786, 22.335831380345102, 0.28, 0.2]
Kyiv ['Kyiv'] [24.680613561528517, 7.34984486592898, 0.26, 0.2]
Santo Domingo ['Santo Domingo'] [26.474501027548357, 7.519601929128936, 0.26, 0.2]
Amaigbo ['Amaigbo'] [42.6608351347212, 22.323035504428674, 0.27, 0.2]
Adelaide ['Adelaide'] [49.6843130999414, 19.538578573455816, 0.26, 0.2]
Tashkent ['Tashkent'] [51.

Dortmund ['Dortmund'] [30.453951156750232, 14.166249296001977, 0.7400000000000002, 0.3]
Minneapolis [Saint Paul] ['Minneapolis [Saint Paul]'] [23.707993515733246, 11.257350479518095, 0.6900000000000002, 0.3]
Lagos ['Lagos'] [69.27175615230232, 17.133075153528182, 0.39, 0.3]
Atlanta ['Atlanta'] [25.1806647711148, 14.247512574144864, 0.6100000000000001, 0.3]
Colombo ['Colombo'] [32.466569709657385, 16.988836483452857, 0.47, 0.3]
Hanoi ['Hanoi'] [62.085605211659406, 28.43791572694092, 0.39999999999999997, 0.3]
Quanzhou ['Quanzhou'] [61.45226674818658, 30.039005278904465, 0.39, 0.3]
Montreal ['Montreal'] [30.870592095101767, 12.663100788632033, 0.41, 0.3]
Mumbai ['Mumbai'] [90.75879796955759, 16.922432715292086, 0.39, 0.3]
Indianapolis ['Indianapolis'] [26.824995630519812, 11.47347261566523, 0.6800000000000002, 0.3]
Kansas City ['Kansas City'] [26.60938452368961, 9.968614881290003, 0.6400000000000001, 0.3]
Riyadh ['Riyadh'] [331.29835599017684, 118.50959960023977, 0.39, 0.3]
New Taipei [Ta

Kozhikode ['Kozhikode'] [30.130055701610807, 14.149048535578828, 0.7100000000000002, 0.3]
Tel Aviv ['Tel Aviv'] [107.07367465461766, 33.67672799778688, 0.39, 0.3]
Recife ['Recife'] [48.51073171871087, 11.293609804876981, 0.39, 0.3]
Ibadan ['Ibadan'] [66.04041746857824, 13.58949561702934, 0.39, 0.3]
Kuwait City ['Kuwait City'] [334.21139520208425, 71.43160333315352, 0.39, 0.3]
Bogura ['Bogura'] [61.27373857636955, 34.917248507416055, 0.53, 0.3]
Goiânia ['Goiânia'] [60.08809047828141, 16.741954766834453, 0.39, 0.3]
Linyi ['Linyi'] [55.089135070154796, 29.078882184789414, 0.41, 0.3]
Pati ['Pati'] [39.14326831748193, 15.635429989521057, 0.5800000000000001, 0.3]
Calgary ['Calgary'] [44.86488765339707, 13.891073555617567, 0.39, 0.3]
Harbin ['Harbin'] [36.42426282208621, 9.01303840066811, 0.38, 0.3]
Zibo ['Zibo'] [46.064531440435914, 19.123801354009146, 0.39, 0.3]
Albuquerque ['Albuquerque'] [120.12739542637973, 33.86154231479796, 0.39, 0.3]
Lisbon ['Lisbon'] [73.27803601963744, 20.5589354639

New Taipei [Taipei] ['New Taipei [Taipei]'] [28.640959286708643, 10.81410600200999, 0.51, 0.4]
Bandung ['Bandung'] [33.79404318058583, 13.995922171719528, 0.51, 0.4]
Orlando ['Orlando'] [46.97853493673659, 19.143744232530295, 0.6400000000000001, 0.4]
Lahore ['Lahore'] [123.58123461987546, 22.12168268205321, 0.52, 0.4]
Cleveland ['Cleveland'] [21.373657067152077, 11.654020362597981, 0.6500000000000001, 0.4]
Portland ['Portland'] [37.6270743490088, 12.158062485636258, 0.6800000000000002, 0.4]
Chennai ['Chennai'] [100.04532258276174, 27.40131406582397, 0.54, 0.4]
Boston ['Boston'] [18.776374849650757, 11.036435037349152, 0.6700000000000002, 0.4]
St. Louis ['St. Louis'] [33.078332276109684, 12.37840421818973, 0.6700000000000002, 0.4]
Naples ['Naples'] [46.22714558942704, 15.591619804781537, 0.52, 0.4]
Singapore ['Singapore'] [61.05050279640749, 18.004882092533183, 0.8000000000000003, 0.4]
Las Vegas ['Las Vegas'] [293.11677505941856, 72.81989686102143, 0.52, 0.4]
Lima ['Lima'] [297.49073427

Calgary ['Calgary'] [51.98875911703894, 15.872638135058805, 0.7200000000000002, 0.4]
Harbin ['Harbin'] [41.44726610197208, 11.387191046645874, 0.51, 0.4]
Zibo ['Zibo'] [52.14336331906948, 22.24020843978495, 0.6400000000000001, 0.4]
Albuquerque ['Albuquerque'] [158.29670051691207, 45.273311559994625, 0.52, 0.4]
Lisbon ['Lisbon'] [93.4140656208581, 27.001269150056448, 0.52, 0.4]
Athens ['Athens'] [113.5447131128588, 25.615601068085773, 0.52, 0.4]
Budapest ['Budapest'] [42.451239847890214, 15.994964260998113, 0.8500000000000003, 0.4]
Havana ['Havana'] [27.56573539995461, 11.734117130105757, 0.7400000000000002, 0.4]
Buffalo ['Buffalo'] [27.82761596226724, 13.915656367874451, 0.6100000000000001, 0.4]
Abidjan ['Abidjan'] [70.28998226868183, 16.945929901428368, 0.52, 0.4]
Kumasi ['Kumasi'] [56.700248556590545, 13.08768863655213, 0.52, 0.4]
San José ['San José'] [42.08042202100888, 16.851961537798623, 0.51, 0.4]
Tripoli ['Tripoli'] [209.8530587390285, 131.53118033694497, 0.52, 0.4]
Warsaw ['Wa

St. Louis ['St. Louis'] [34.07623252367423, 12.37840421818973, 0.67, 0.5]
Naples ['Naples'] [49.78231353440668, 16.535029293271876, 0.76, 0.5]
Singapore ['Singapore'] [66.95491988609078, 18.004882092533183, 0.8, 0.5]
Las Vegas ['Las Vegas'] [366.3239432679559, 89.4150802616883, 0.64, 0.5]
Lima ['Lima'] [371.45357522188203, 90.73082141551859, 0.64, 0.5]
San Antonio ['San Antonio'] [78.5552815849095, 23.266917847458206, 0.65, 0.5]
Hyderabad ['Hyderabad'] [161.64234378735077, 39.745934085728535, 0.79, 0.5]
Columbus ['Columbus'] [30.373882079584078, 12.111243355374837, 0.66, 0.5]
Accra ['Accra'] [109.42913491834467, 43.41219454355527, 0.7400000000000001, 0.5]
Wuhan ['Wuhan'] [81.08845754271377, 17.334062829775597, 0.78, 0.5]
Bradenton ['Bradenton'] [40.55046580103848, 12.302789018753193, 0.67, 0.5]
Sacramento ['Sacramento'] [108.43260537249266, 26.979176236184415, 0.68, 0.5]
Chattogram ['Chattogram'] [37.58612193421178, 11.510595868345034, 0.8, 0.5]
Baghdad ['Baghdad'] [234.05110478269194,

San José ['San José'] [44.43331936708684, 17.86032866568261, 0.75, 0.5]
Tripoli ['Tripoli'] [262.3042990729949, 162.23419933531488, 0.64, 0.5]
Warsaw ['Warsaw'] [31.306252600443152, 11.684830900723055, 0.65, 0.5]
Tunis ['Tunis'] [218.75222953104975, 59.69646268641378, 0.64, 0.5]
Concord ['Concord'] [117.3750319473188, 28.173004090248924, 0.76, 0.5]
Changchun ['Changchun'] [48.22992196489279, 9.575572789661104, 0.6900000000000001, 0.5]
Tucson ['Tucson'] [226.32035194038508, 69.23767916797557, 0.64, 0.5]
Omaha ['Omaha'] [36.132877338046896, 13.684727477278889, 0.65, 0.5]
Faisalabad ['Faisalabad'] [141.7365949770784, 30.69966154770625, 0.67, 0.5]
Providence ['Providence'] [16.679805285436323, 9.726394097689337, 0.63, 0.5]
Kano ['Kano'] [225.95889098892704, 52.2172625450175, 0.64, 0.5]
Casablanca ['Casablanca'] [213.4045620073002, 47.34667585872328, 0.64, 0.5]
Vienna ['Vienna'] [43.543981693381795, 12.008429042310247, 0.7400000000000001, 0.5]
Allentown ['Allentown'] [12.949342770181145, 5.

In [30]:
dff.to_csv("../data_revision/results/allCities1.csv", index=None)